In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import edward2 as ed
import warnings

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow_probability.python.distributions import distribution
from tensorflow_probability.python.internal import distribution_util
from tensorflow_probability.python.internal import dtype_util
from tensorflow_probability.python.internal import reparameterization

In [16]:
class Constant(distribution.Distribution):
    def __init__(self, name="Constant"):        
        super(Constant, self).__init__(name=name, 
                                       dtype=tf.float64,
                                       validate_args=False,
                                       allow_nan_stats=True,
                                       reparameterization_type=reparameterization.NOT_REPARAMETERIZED)

    def _log_prob(self, x):
        return 0

    def _log_cdf(self, x):
        return 0
    
    def _sample_n(self, n, seed=None):
        return tf.zeros(shape=n)

In [87]:
def model():
    x = ed.RandomVariable(Constant())
    y = ed.Normal(x, 1, name="y")

In [88]:
def joint(x, y):
    log_joint =  ed.make_log_joint_fn(model) 
    return log_joint(x=x, y=y)

In [92]:
num_results = 5000
num_burnin_steps = 3000

states, kernel_results = tfp.mcmc.sample_chain(
    num_results=num_results,
    num_burnin_steps=num_burnin_steps,
    current_state=[
        tf.zeros([], name='x1'),
        tf.zeros([], name='y1')
    ],
    kernel=tfp.mcmc.Me())

TypeError: __init__() missing 1 required positional argument: 'target_log_prob_fn'

In [72]:
states, kernel_results = tfp.mcmc.sample_chain(
    num_results=100,
    num_burnin_steps=100,
    current_state=[
        tf.zeros([], name='init_avg_effect')
    ],
    kernel=tfp.mcmc.MetropolisHastings(
    tfp.mcmc.UncalibratedHamiltonianMonteCarlo(
        target_log_prob_fn=lambda x: -x - x**2,
        step_size=0.1,
        num_leapfrog_steps=3)))

/home/simon/miniconda3/envs/tensorflow/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


In [73]:
init_avg_effect  = states

In [74]:
with tf.Session() as sess:
  [
         init_avg_effect_,
      is_accepted_,
  ] = sess.run([
      init_avg_effect,     
      kernel_results.is_accepted
  ])

In [75]:
init_avg_effect_

[array([-0.57153326, -0.42169446, -0.21875943, -0.21651788, -0.5240787 ,
        -0.61782634, -0.6855886 , -0.8629459 , -1.3405784 , -1.0971452 ,
        -1.2420467 , -0.86934066, -0.49931645, -0.3757949 ,  0.11800686,
         0.1409323 , -0.05767936, -0.13410403, -0.22536881, -0.4676463 ,
        -0.140457  , -0.11303421, -0.12927218, -0.3607694 ,  0.2757895 ,
         0.42137575, -0.37745687, -0.25776744, -0.01182785, -0.2177423 ,
        -0.496884  , -0.15023685, -0.69865286, -0.9575068 , -0.9502926 ,
        -0.76831824, -0.72301507, -0.6610129 , -0.5886413 , -0.61074877,
        -0.68204176, -0.5844968 , -0.41008902, -0.64495814, -1.0563143 ,
        -0.83192503, -0.97529376, -0.6941219 ,  0.15230492,  0.07676874,
         0.37041733,  0.0668562 ,  0.06954142,  0.298021  ,  0.1578378 ,
         0.61161387,  0.59987307,  0.62318325,  1.0484128 ,  0.6270744 ,
         0.5257303 ,  0.7160263 ,  0.92825025,  0.30353218,  0.1072963 ,
         0.23547137,  0.5468046 ,  0.7477072 ,  0.6